Our Task: Analyze Spam Emails (https://www.dataquest.io/blog/regular-expressions-data-scientists/)
In this tutorial, we'll use the Fraudulent Email Corpus from Kaggle. It contains thousands of phishing emails sent between 1998 and 2007. They're pretty entertaining to read.

You can find the full corpus here (https://www.kaggle.com/datasets/rtatman/fraudulent-email-corpus). But we'll start by learning basic regex commands using a few emails. If you'd like, you can use our test file as well, or you can try this with the full corpus.

# Introducing Python's Regex Module
First, we'll prepare the data set by opening the test file, setting it to read-only, and reading it. We'll also assign it to a variable, fh (for "file handle").

In [1]:
fh = open(r"test_emails.txt", "r").read()

Notice that we precede the directory path with an r. This technique converts a string into a raw string, which helps to avoid conflicts caused by how some machines read characters, such as backslashes in directory paths on Windows.

Now, suppose we want to find out who the emails are from. We could try raw Python on its own:

for line in fh.split("n"):
    if "From:" in line:
        print(line)

But that's not giving us exactly what we want. If you take a look at our test file, we could figure out why and fix it, but instead, let's use Python's re module and do it with regular expressions!

We'll start by importing Python's re module. Then, we'll use a function called re.findall() that returns a list of all instances of a pattern we define in the string we're looking at.

Here's how it looks:

In [2]:
import re

for line in re.findall("From:.*", fh):
    print(line)

From: "Mr. Ben Suleman" <bensul2004nng@spinfinder.com>
From: "PRINCE OBONG ELEME" <obong_715@epatra.com>


This is essentially the same length as our raw Python, but that's because it's a very simple example. The more you're trying to do, the more effort Python regex is likely to save you.

Before we move on, let's take a closer look at re.findall(). This function takes two arguments in the form of re.findall(pattern, string). Here, pattern represents the substring we want to find, and string represents the main string we want to find it in. The main string can consist of multiple lines. In this case, we're having it search through all of fh, the file with our selected emails.

The .* is a shorthand for a string pattern. Regular expressions work by using these shorthand patterns to find specific patterns in text, so let's take a look at some other common examples:

# Common Python Regex Patterns
The pattern we used with re.findall() above contains a fully spelled-out out string, "From:". This is useful when we know precisely what we're looking for, right down to the actual letters and whether or not they're upper or lower case. If we don't know the exact format of the strings we want, we'd be lost. Fortunately, regex has basic patterns that account for this scenario. Let's look at the ones we use in this tutorial:

w matches alphanumeric characters, which means a-z, A-Z, and 0-9. It also matches the underscore, _, and the dash, -.
d matches digits, which means 0-9.
s matches whitespace characters, which include the tab, new line, carriage return, and space characters.
S matches non-whitespace characters.
. matches any character except the new line character n.
With these regex patterns in hand, you'll quickly understand our code above as we go on to explain it.

# Working with Regex Patterns
We can now explain the use of .* in the line re.findall("From:.*", text) above. Let's look at . first:

In [3]:
for line in re.findall("From:.", fh):
    print(line)

From: 
From: 


By adding a . next to From:, we look for one additional character next to it. Because . looks for any character except n, it captures the space character, which we cannot see. We can try more dots to verify this.

In [4]:
for line in re.findall("From:...........", fh):
    print(line)

From: "Mr. Ben S
From: "PRINCE OB


It looks like adding dots does acquire the rest of the line for us. But, it's tedious and we don't know how many dots to add. This is where the asterisk symbol, *, comes in.

* matches zero or more instances of a pattern on its left. This means it looks for repeating patterns. When we look for repeating patterns, we say that our search is "greedy." If we don't look for repeating patterns, we can call our search "non-greedy" or "lazy."

Let's construct a greedy search for . with *.

In [5]:
for line in re.findall("From:.*", fh):
    print(line)

From: "Mr. Ben Suleman" <bensul2004nng@spinfinder.com>
From: "PRINCE OBONG ELEME" <obong_715@epatra.com>


Because * matches zero or more instances of the pattern indicated on its left, and . is on its left here, we are able to acquire all the characters in the From: field until the end of the line. This prints out the full line with beautifully succinct code.

We might even go further and isolate only the name. Let's use re.findall() to return a list of lines containing the pattern "From:.*" as we've done before. We'll assign it to the variable match for neatness. Next, we'll iterate through the list. In each cycle, we'll execute re.findall again, matching the first quotation mark to pick out just the name:

In [6]:
match = re.findall("From:.*", fh)

for line in match:
    print(re.findall('\".*\"', line))

['"Mr. Ben Suleman"']
['"PRINCE OBONG ELEME"']


Notice that we use a backslash next to the first quotation mark. The backslash is a special character used for escaping other special characters. For instance, when we want to use a quotation mark as a string literal instead of a special character, we escape it with a backslash like this: \". If we do not escape the pattern above with backslashes, it would become "".*"", which the Python interpreter would read as a period and an asterisk between two empty strings. It would produce an error and break the script. Hence, it's crucial that we escape the quotation marks here with backslashes.

After the first quotation mark is matched, .* acquires all the characters in the line until the next quotation mark, also escaped in the pattern. This gets us just the name, within quotation marks. The name is also printed within square brackets because re.findall returns matches in a list.

What if we want the email address instead?

In [7]:
match = re.findall("From:.*", fh)

for line in match:
    print(re.findall("\w\S*@*.\w", line))

['From', 'Mr. B', 'en S', 'uleman', 'bensul2004nng@spinfinder.com']
['From', 'PRINCE O', 'BONG E', 'LEME', 'obong_715@epatra.com']


Looks simple enough, doesn't it? Only the pattern is different. Let's walk through it.

Here's how we match just the front part of the email address:

In [8]:
for line in match:
    print(re.findall("\w\S*@", line))

['bensul2004nng@']
['obong_715@']


Emails always contain an @ symbol, so we start with it. The part of the email before the @ symbol might contain alphanumeric characters, which means w is required. However, because some emails contain a period or a dash, that's not enough. We add S to look for non-whitespace characters. But, w\S will get only two characters. Add * to look for repetitions. The front part of the pattern thus looks like this: \w\S*@.

Now for the pattern behind the @ symbol:

In [9]:
for line in match:
    print(re.findall("@.*", line))

['@spinfinder.com>']
['@epatra.com>']


The domain name usually contains alphanumeric characters, periods, and a dash sometimes, so a . will do. To make it greedy, we extend the search with a *. This allows us to match any character till the end of the line.

If we look at the line closely, we see that each email is encapsulated within angle brackets, < and >. Our pattern, .*, includes the closing bracket, >. Let's remedy it:

In [10]:
for line in match:
    print(re.findall("@.*\w", line))

['@spinfinder.com']
['@epatra.com']


Email addresses end with an alphanumeric character, so we cap the pattern with w. So, after the @ symbol we have .*\w, which means that the pattern we want is a group of any type of characters ending with an alphanumeric character. This excludes >.

Our full email address pattern thus looks like this: \w\S*@.*\w.

Phew! That was quite a bit to work through. Next, we'll run through some common re functions that will be useful when we start reorganizing our corpus.

# Common Python Regex Functions
re.findall() is undeniably useful, but it's not the only built-in function that's available to us in re:

re.search()
re.split()
re.sub()
Let's look at these one by one before using them to bring some order to our data set.

# re.search()
While re.findall() matches all instances of a pattern in a string and returns them in a list, re.search() matches the first instance of a pattern in a string, and returns it as a re match object.

In [11]:
match = re.search("From:.*", fh)
print(type(match))
print(type(match.group()))
print(match)
print(match.group())

<class 're.Match'>
<class 'str'>
<re.Match object; span=(204, 258), match='From: "Mr. Ben Suleman" <bensul2004nng@spinfinder>
From: "Mr. Ben Suleman" <bensul2004nng@spinfinder.com>


Like re.findall(), re.search() also takes two arguments. The first is the pattern to match, and the second is the string to find it in. Here, we've assigned the results to the match variable for neatness.

Because re.search() returns a re match object, we can't display the name and email address by printing it directly. Instead, we have to apply the group() function to it first. We've printed both their types out in the code above. As we can see, group() converts the match object into a string.

We can also see that printing match displays properties beyond the string itself, whereas printing match.group() displays only the string.

# re.split()
Suppose we need a quick way to get the domain name of the email addresses. We could do it with three regex operations, like so:

In [12]:
address = re.findall("From:.*", fh)
for item in address:
    for line in re.findall("\w\S*@.*\w", item):
        username, domain_name = re.split("@", line)
        print("{}, {}".format(username, domain_name))

bensul2004nng, spinfinder.com
obong_715, epatra.com


The first line is familiar. We return a list of strings, each containing the contents of the From: field, and assign it to a variable. Next, we iterate through the list to find the email addresses. At the same time, we iterate through the email addresses and use the re module's split() function to snip each address in half, with the @ symbol as the delimiter. Finally, we print it.

# re.sub()
Another handy re function is re.sub(). As the function name suggests, it substitutes parts of a string. An example:

In [13]:
sender = re.search("From:.*", fh)
address = sender.group()
email = re.sub("From", "Email", address)
print(address)
print(email)

From: "Mr. Ben Suleman" <bensul2004nng@spinfinder.com>
Email: "Mr. Ben Suleman" <bensul2004nng@spinfinder.com>


We've already seen the tasks on the first and second lines before. On the third line, we apply re.sub() on address, which is the full From: field in the email header.

re.sub() takes three arguments. The first is the substring to substitute, the second is a string we want in its place, and the third is the main string itself.

# Regex with Pandas
Now we have the basics of Python regex in hand. But often for data tasks, we're not actually using raw Python, we're using the pandas library. Now let's take our regex skills to the next level by bringing them into a pandas workflow.

In [14]:
import re
import pandas as pd
import email

emails = []

fh = open(r"test_emails.txt", "r").read()

contents = re.split(r"From r",fh)
contents.pop(0)

for item in contents:
    emails_dict = {}

    sender = re.search(r"From:.*", item)

    if sender is not None:
        s_email = re.search(r"\w\S*@.*\w", sender.group())
        s_name = re.search(r":.*<", sender.group())
    else:
        s_email = None
        s_name = None

    if s_email is not None:
        sender_email = s_email.group()
    else:
        sender_email = None

    emails_dict["sender_email"] = sender_email

    if s_name is not None:
        sender_name = re.sub("\s*<", "", re.sub(":\s*", "", s_name.group()))
    else:
        sender_name = None

    emails_dict["sender_name"] = sender_name

    recipient = re.search(r"To:.*", item)

    if recipient is not None:
        r_email = re.search(r"\w\S*@.*\w", recipient.group())
        r_name = re.search(r":.*<", recipient.group())
    else:
        r_email = None
        r_name = None

    if r_email is not None:
        recipient_email = r_email.group()
    else:
        recipient_email = None
    emails_dict["recipient_email"] = recipient_email

    if r_name is not None:
        recipient_name = re.sub("s*<", "", re.sub(":s*", "", r_name.group()))
    else:
        recipient_name = None

    emails_dict["recipient_name"] = recipient_name

    date_field = re.search(r"Date:.*", item)

    if date_field is not None:
        date = re.search(r"\d+\s\w+\s\d+", date_field.group())
    else:
        date = None

    if date is not None:
        date_sent = date.group()
    else:
        date_sent = None

    emails_dict["date_sent"] = date_sent

    subject_field = re.search(r"Subject: .*", item)

    if subject_field is not None:
        subject = re.sub(r"Subject: ", "", subject_field.group())
    else:
        subject = None

    emails_dict["subject"] = subject

    # "item" substituted with "email content here" so full email not
# displayed.

    full_email = email.message_from_string(item)
    body = full_email.get_payload()
    emails_dict["email_body"] = "email body here"

    emails.append(emails_dict)
# Print number of dictionaries, and hence, emails, in the list.
print("Number of emails: " + str(len(emails_dict)))

print("n")

# Print first item in the emails list to see how it looks.
for key, value in emails[0].items():
    print(str(key) + ": " + str(emails[0][key]))

Number of emails: 7
n
sender_email: bensul2004nng@spinfinder.com
sender_name: "Mr. Ben Suleman"
recipient_email: R@M
recipient_name: None
date_sent: 31 Oct 2002
subject: URGENT ASSISTANCE /RELATIONSHIP (P)
email_body: email body here


In [15]:
import pandas as pd
# Module imported above, imported again as reminder.
emails_df = pd.DataFrame(emails)

In [16]:
pd.DataFrame.head(emails_df, n=3)

,sender_email,sender_name,recipient_email,recipient_name,date_sent,subject,email_body
0,bensul2004nng@spinfinder.com,"""Mr. Ben Suleman""",R@M,None,31 Oct 2002,URGENT ASSISTANCE /RELATIONSHIP (P),email body here
1,obong_715@epatra.com,"""PRINCE OBONG ELEME""",obong_715@epatra.com,None,31 Oct 2002,GOOD DAY TO YOU,email body here


In [17]:
emails_df[emails_df["sender_email"].str.contains("epatra|spinfinder")]

,sender_email,sender_name,recipient_email,recipient_name,date_sent,subject,email_body
0,bensul2004nng@spinfinder.com,"""Mr. Ben Suleman""",R@M,None,31 Oct 2002,URGENT ASSISTANCE /RELATIONSHIP (P),email body here
1,obong_715@epatra.com,"""PRINCE OBONG ELEME""",obong_715@epatra.com,None,31 Oct 2002,GOOD DAY TO YOU,email body here


In [18]:
# Step 1: find the index where the "sender_email" column contains "@spinfinder.com".
index = emails_df[emails_df["sender_email"].str.contains(r"\w\S*@spinfinder.com")].index.values

In [19]:
# Step 2: use the index to find the value of the cell i the "sender_email" column.
# The result is returned as pandas Series object
address_Series = emails_df.loc[index]["sender_email"]
print(address_Series)
print(type(address_Series))

0    bensul2004nng@spinfinder.com
Name: sender_email, dtype: object
<class 'pandas.core.series.Series'>


In [20]:
# Step 3: extract the email address, which is at index 0 in the Series object.
address_string = address_Series[0]
print(address_string)
print(type(address_string))

bensul2004nng@spinfinder.com
<class 'str'>


In [21]:
# Step 4: find the value of the "email_body" column where the "sender email" column is address_string.
print(emails_df[emails_df["sender_email"] == address_string]["email_body"].values)

['email body here']
